# Facebook爬蟲

In [1]:
import pandas as pd
import re, time, requests, datetime, gc
from selenium import webdriver
from bs4 import BeautifulSoup

# 定義函數供後續使用

在這裡我們先定義一個函數，希望把網頁中各篇貼文的連結都找出來!
ulr放我們要爬的Facebook網址，n是稍後要送出幾次滾動網頁到底部的命令，藉以加載更多資料。

In [2]:
def FindLinks(url, n):
    Links = []
    driver.get(url)
    for i in range(n):
        time.sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
    driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    soup = BeautifulSoup(driver.page_source)
    posts = soup.findAll('div', {'class':'clearfix y_c3pyo2ta3'})
    for i in posts:
        Links.append('https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0])
    return Links

定義一個展開所有留言的函數，透過while迴圈反覆搜尋與點擊「看更多留言」、「看更多回覆」與「看完整貼文內容」等按鈕。
在過程中會出現請我們登入或註冊的彈跳視窗，但我們不確定到底什麼時候會跳出，因此需要在過程中反覆偵測是有出現這個彈窗，若有就點擊「Not Now」

In [3]:
# 展開url中的所有留言 與 留言的留言
def expand(url):
    driver.get(url)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    try:
        driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
        time.sleep(1)
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(1)
        driver.find_element_by_id('expanding_cta_close_button').click() 
    except:
        time.sleep(1)
    # 偵測是否有「更多留言」（第一層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                driver.find_element_by_id('expanding_cta_close_button').click()
            except:
                time.sleep(0.5)            
            try:
                i.click()
            except:
                time.sleep(0.5)   
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))>0:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                driver.find_element_by_id('expanding_cta_close_button').click()
            except:
                time.sleep(0.5)
            try:
                i.click()
            except:
                time.sleep(0.5)
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                i.click()
            except:
                time.sleep(0.5)

透過逐層搜索的方式，逐步定位我們要找的資訊
在這個環節需要反覆透過Chrome的功能比對資料，需要花一些心力進行比對
另外在這部分也使用了大量個try-except，原因是許多資料是有內容才會出現。例如並非每天貼人都會收到「哈哈」、「生氣」的心情。

In [4]:
# 文章內容與互動摘要
def CrawlPost(soup):
    time.sleep(1)
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # po文人資訊區塊
    PosterInfo = userContent.find('div', {'class':'l_c3pyo2v0u i_c3pynyi2f clearfix'})
    # 互動摘要區(讚、留言與分享)
    feedback = soup.find('form', {'class':'commentable_item collapsed_comments'})
    # 名稱
    Name = PosterInfo.find('img').attrs['aria-label']
    # ID
    ID = PosterInfo.find('a', {'class':'_5pb8 o_c3pynyi2g _8o _8s lfloat _ohe'}).attrs['href'].split('/?',2)[0].split('/',-1)[-1]
    # 網址
    Link = driver.current_url
    # 發文時間
    try:
        Time = PosterInfo.find('abbr').attrs['title']
    except:
        Time = PosterInfo.find('div', {'class':'_1atc fsm fwn fcg'}).text
    # 文章內容
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = ""
    # Like
    try:
        Like = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Like = '0' 
    # Love
    try:
        Love = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Love = '0' 
    # Haha
    try:
        Haha = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Haha = '0' 
    # Wow
    try:
        Wow = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Wow = '0' 
    # Sad
    try:
        Sad = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Sad = '0' 
    # Angry
    try:
        Angry = feedback.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label'].split(' ',2)[0]
    except:
        Angry = '0'
    # 留言
    try:
        Comments = feedback.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
    except:
        Comments = '0' 
    # 分享
    try:
        Share = feedback.find('span', {'class':'_355t _4vn2'}).text.split(' ',2)[0]
    except:
        Share = '0' 
    Updatetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return pd.DataFrame(
        data = [{'Name':Name,
                 'ID':ID,
                 'Link':Link,
                 'Time':Time,
                 'Content':Content,
                 'Like':Like,
                 'Love':Love,
                 'Haha':Haha,
                 'Wow':Wow,
                 'Sad':Sad,
                 'Angry':Angry,
                 'Comments':Comments,
                 'Share':Share,
                 'Updatetime':Updatetime}],
        columns = ['Name', 'ID', 'Time', 'Content', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Comments', 'Share', 'Link', 'Updatetime'])

這邊要留意雖然都是粉絲留言，但實際上分成「回應貼文的留言」與「回應留言的留言」。
函數中的第一個迴圈是用來抓「回應貼文的留言」，第二個則是抓「回應留言的留言」。讀者可以自行比較一下兩個迴圈中不同的地方。

In [5]:
# 留言
def CrawlComment(soup):
    time.sleep(1)
    Comments = pd.DataFrame()
    # po文區塊
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # 用戶留言區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    # 回應貼文的留言
    for i in userContent.findAll('div', {'data-testid':'UFI2Comment/root_depth_0'}):
        try:
            Content = i.find('span', {'dir':'ltr'}).text
        except:
            Content = 'sticker'
        Comment = pd.DataFrame(data = [{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                 'Name':i.find('img').attrs['alt'],
                                 'Time':i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'],
                                 'Content':Content,
                                 'Link':driver.current_url}],
                        columns = ['ID', 'Name', 'Time', 'Content', 'Link'])
        Comments = pd.concat([Comments, Comment], ignore_index=True)
    
    # 回應留言的留言
    for i in userContent.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
        try:
            Content = i.find('span', {'dir':'ltr'}).text
        except:
            Content = 'sticker'
        Comment = pd.DataFrame(data = [{'ID':i.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                 'Name':i.find('img').attrs['alt'],
                                 'Time':i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'],
                                 'Content':Content,
                                 'Link':driver.current_url}],
                        columns = ['ID', 'Name', 'Time', 'Content', 'Link'])
        Comments = pd.concat([Comments, Comment], ignore_index=True)
    Comments['Updatetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    return Comments

# 組合應用
組合以上函數，針對逐一爬取List中的貼文，依據貼文的類型做後續處理

- 新貼文：爬取貼文與留言
- 舊貼文但有新留言：更新貼文摘要並爬取留言
- 舊貼文且無新留言：pass
- 其他：查看情況

In [7]:
# https://www.facebook.com/cathayunitedbank/
# https://www.facebook.com/tstartel/
# https://www.facebook.com/chtmobile/
# https://www.facebook.com/taiwanmobile/
# https://www.facebook.com/fareastone/
# https://www.facebook.com/Aptg.tw/
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
time.sleep(1.5)
driver.find_element_by_xpath('//a[@class="_sv4"]').click()

In [11]:
# 載入先前資料
Posts = pd.read_csv('C:/Users/TL_Yu/Desktop/Posts.csv',
                    skiprows=[0],
                    names = ['Name', 'ID', 'Time', 'Content', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Comments', 'Share', 'Link', 'Updatetime'],
                    dtype='str')
Comments = pd.read_csv('C:/Users/TL_Yu/Desktop/Comments.csv',
                       skiprows=[0],
                       names=['Content', 'ID', 'Link', 'Name', 'Time', 'Updatetime'],
                       dtype='str')

#Posts = pd.DataFrame()
#Comments = pd.DataFrame()

## 台星

In [12]:
url = 'https://www.facebook.com/tstartel/'
Links = FindLinks(url = url, n = 15)
# 抓下來所有留言
for i in Links:
    print('Dealing with: ' + i)
    try:
        driver.get(i)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
        nPost = CrawlPost(soup)
        compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
                           right = Posts.loc[:,['Link', 'Comments']],
                           how = 'inner',
                           on = ['Link'],
                           indicator=True,
                           suffixes=('_n', '_o'))
        # 新貼文直接爬留言
        if len(compare) == 0:
            print('Type: ' + 'New Post')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
        # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
        elif compare.Comments_n.values > compare.Comments_o.values:
            # 開始爬留言並將文章摘要匯入
            print('Type: ' + 'Old Post with New Comments')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
            nComments = pd.merge(left = nComments,
                                 right = Comments,
                                 how = 'left',
                                 on = ['ID', 'Name', 'Content', 'Link'],
                                 suffixes=('', '_o'),
                                 indicator=True)
            nComments = nComments.loc[nComments._merge == 'left_only',:]
            nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
        # 舊文章也無新留言，pass過去
        elif compare.Comments_n.values == compare.Comments_o.values:
            print('Type: ' + 'Old Post with Old Comments')
            nComments = pd.DataFrame()
        # 其他狀況，請釐清問題
        else:
            print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
            nComments = pd.DataFrame()        
        
        # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
        Posts = pd.concat([Posts, nPost], ignore_index=True)
        Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
        Posts = Posts.drop_duplicates(subset ="Link",
                                      keep= 'first',
                                      inplace = False) 
        Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
        print('Result: Succed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
    except:
        print('Result: Failed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv',index = False)
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv',index = False)

Dealing with: https://www.facebook.com/tstartel/videos/321955001812559/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:14:47
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/posts/3269351643090860
Type: Uncertain condiction! Plz Check this website:...
Result: Succed!
Time Log: 2019-05-15 22:14:51
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/posts/3267141889978502
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:15:17
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3259789660713725/
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:15:32
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3264107533615271/
Type: Old Post with Old Comments
Result:

Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:18:50
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.854369967922385/3197740693585289/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:18:53
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/posts/3194840400541985
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:18:57
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/photos/a.2482674201758612/3192347447457947/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:19:01
----------------------------------------------------------


Dealing with: https://www.facebook.com/tstartel/videos/1285036481654086/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:19:05
----------------------------------

In [13]:
gc.collect()

12858

## 中華

In [14]:
url = 'https://www.facebook.com/chtmobile/'
Links = FindLinks(url = url, n = 15)
# 抓下來所有留言
for i in Links:
    print('Dealing with: ' + i)
    try:
        driver.get(i)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
        nPost = CrawlPost(soup)
        compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
                           right = Posts.loc[:,['Link', 'Comments']],
                           how = 'inner',
                           on = ['Link'],
                           indicator=True,
                           suffixes=('_n', '_o'))
        # 新貼文直接爬留言
        if len(compare) == 0:
            print('Type: ' + 'New Post')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
        # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
        elif compare.Comments_n.values > compare.Comments_o.values:
            # 開始爬留言並將文章摘要匯入
            print('Type: ' + 'Old Post with New Comments')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
            nComments = pd.merge(left = nComments,
                                 right = Comments,
                                 how = 'left',
                                 on = ['ID', 'Name', 'Content', 'Link'],
                                 suffixes=('', '_o'),
                                 indicator=True)
            nComments = nComments.loc[nComments._merge == 'left_only',:]
            nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
        # 舊文章也無新留言，pass過去
        elif compare.Comments_n.values == compare.Comments_o.values:
            print('Type: ' + 'Old Post with Old Comments')
            nComments = pd.DataFrame()
        # 其他狀況，請釐清問題
        else:
            print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
            nComments = pd.DataFrame()        
        
        # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
        Posts = pd.concat([Posts, nPost], ignore_index=True)
        Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
        Posts = Posts.drop_duplicates(subset ="Link",
                                      keep= 'first',
                                      inplace = False) 
        Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
        print('Result: Succed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
    except:
        print('Result: Failed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv',index = False)
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv',index = False)

Dealing with: https://www.facebook.com/chtmobile/videos/2758708034143708/
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:21:21
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2715269541836004/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:21:24
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2707938675902424/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:21:28
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/posts/2703335766362715
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:21:31
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/posts/2695733070456318
Type: Old Post with Old Comments
Resul

Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:23:21
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2624215924274700/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:23:24
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2613377002025259/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:23:28
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2613255065370786/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:23:31
----------------------------------------------------------


Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2614263715269921/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22

In [15]:
gc.collect()

66960

## 台哥大

In [16]:
url = 'https://www.facebook.com/taiwanmobile/'
Links = FindLinks(url = url, n = 15)
# 抓下來所有留言
for i in Links:
    print('Dealing with: ' + i)
    try:
        driver.get(i)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
        nPost = CrawlPost(soup)
        compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
                           right = Posts.loc[:,['Link', 'Comments']],
                           how = 'inner',
                           on = ['Link'],
                           indicator=True,
                           suffixes=('_n', '_o'))
        # 新貼文直接爬留言
        if len(compare) == 0:
            print('Type: ' + 'New Post')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
        # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
        elif compare.Comments_n.values > compare.Comments_o.values:
            # 開始爬留言並將文章摘要匯入
            print('Type: ' + 'Old Post with New Comments')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
            nComments = pd.merge(left = nComments,
                                 right = Comments,
                                 how = 'left',
                                 on = ['ID', 'Name', 'Content', 'Link'],
                                 suffixes=('', '_o'),
                                 indicator=True)
            nComments = nComments.loc[nComments._merge == 'left_only',:]
            nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
        # 舊文章也無新留言，pass過去
        elif compare.Comments_n.values == compare.Comments_o.values:
            print('Type: ' + 'Old Post with Old Comments')
            nComments = pd.DataFrame()
        # 其他狀況，請釐清問題
        else:
            print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
            nComments = pd.DataFrame()        
        
        # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
        Posts = pd.concat([Posts, nPost], ignore_index=True)
        Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
        Posts = Posts.drop_duplicates(subset ="Link",
                                      keep= 'first',
                                      inplace = False) 
        Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
        print('Result: Succed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
    except:
        print('Result: Failed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv',index = False)
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv',index = False)

Dealing with: https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2473289892891437/
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:26:48
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2474118609475232/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:26:55
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/posts/2474400126113747
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:27:03
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/videos/606775269838497/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:27:07
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/posts/2471318653088561
Type: Old Post with New Comments
Result: Failed!
Time Log:

Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:29:32
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/posts/2460245490862544
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:29:35
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/posts/2460199744200452
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:29:39
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/photos/a.1448377238716046/2459996730887420/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:29:43
----------------------------------------------------------


Dealing with: https://www.facebook.com/taiwanmobile/posts/2459393190947774
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:29:47
----------------------------------------

In [17]:
gc.collect()

87006

## 遠傳

In [18]:
url = 'https://www.facebook.com/fareastone/'
Links = FindLinks(url = url, n = 15)
# 抓下來所有留言
for i in Links:
    print('Dealing with: ' + i)
    try:
        driver.get(i)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
        nPost = CrawlPost(soup)
        compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
                           right = Posts.loc[:,['Link', 'Comments']],
                           how = 'inner',
                           on = ['Link'],
                           indicator=True,
                           suffixes=('_n', '_o'))
        # 新貼文直接爬留言
        if len(compare) == 0:
            print('Type: ' + 'New Post')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
        # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
        elif compare.Comments_n.values > compare.Comments_o.values:
            # 開始爬留言並將文章摘要匯入
            print('Type: ' + 'Old Post with New Comments')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
            nComments = pd.merge(left = nComments,
                                 right = Comments,
                                 how = 'left',
                                 on = ['ID', 'Name', 'Content', 'Link'],
                                 suffixes=('', '_o'),
                                 indicator=True)
            nComments = nComments.loc[nComments._merge == 'left_only',:]
            nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
        # 舊文章也無新留言，pass過去
        elif compare.Comments_n.values == compare.Comments_o.values:
            print('Type: ' + 'Old Post with Old Comments')
            nComments = pd.DataFrame()
        # 其他狀況，請釐清問題
        else:
            print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
            nComments = pd.DataFrame()        
        
        # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
        Posts = pd.concat([Posts, nPost], ignore_index=True)
        Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
        Posts = Posts.drop_duplicates(subset ="Link",
                                      keep= 'first',
                                      inplace = False) 
        Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
        print('Result: Succed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
    except:
        print('Result: Failed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv',index = False)
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv',index = False)

Dealing with: https://www.facebook.com/fareastone/photos/a.163720023641308/2536222723057681/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:33:22
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/photos/a.163720023641308/2535914766421810/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:33:37
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/videos/328924844668284/
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:33:48
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/posts/2527683350578285
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:34:02
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/photos/a.163720023641308/2527650693914884/
Type: Old Post with Old Comments
Result: Succed!
T

Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:36:57
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/posts/2500004880012799
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:37:01
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/photos/a.163720023641308/2498842046795749/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:37:05
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/posts/2498841476795806
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:37:08
----------------------------------------------------------


Dealing with: https://www.facebook.com/fareastone/photos/a.163720023641308/2490071891006098/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:37:12
-----------------------------

In [19]:
gc.collect()

26392

## 亞太

In [20]:
url = 'https://www.facebook.com/Aptg.tw/'
Links = FindLinks(url = url, n = 30)
# 抓下來所有留言
for i in Links:
    print('Dealing with: ' + i)
    try:
        driver.get(i)
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source)
        # 檢視文章摘要，分成全新文章、舊文章但有新留言、舊文章也無新留言
        nPost = CrawlPost(soup)
        compare = pd.merge(left = nPost.loc[:,['Link', 'Comments']] ,
                           right = Posts.loc[:,['Link', 'Comments']],
                           how = 'inner',
                           on = ['Link'],
                           indicator=True,
                           suffixes=('_n', '_o'))
        # 新貼文直接爬留言
        if len(compare) == 0:
            print('Type: ' + 'New Post')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
        # 舊文章但有新留言，僅保留新的留言(drop掉已經在資料庫的資料)
        elif compare.Comments_n.values > compare.Comments_o.values:
            # 開始爬留言並將文章摘要匯入
            print('Type: ' + 'Old Post with New Comments')
            expand(i)
            soup = BeautifulSoup(driver.page_source)
            nComments = CrawlComment(soup)
            nComments = pd.merge(left = nComments,
                                 right = Comments,
                                 how = 'left',
                                 on = ['ID', 'Name', 'Content', 'Link'],
                                 suffixes=('', '_o'),
                                 indicator=True)
            nComments = nComments.loc[nComments._merge == 'left_only',:]
            nComments = nComments.drop(['Time_o', 'Updatetime_o', '_merge'], axis=1)
        # 舊文章也無新留言，pass過去
        elif compare.Comments_n.values == compare.Comments_o.values:
            print('Type: ' + 'Old Post with Old Comments')
            nComments = pd.DataFrame()
        # 其他狀況，請釐清問題
        else:
            print('Type: ' + 'Uncertain condiction! Plz Check this website:' + '...')
            nComments = pd.DataFrame()        
        
        # post的資料直接concat在最下面，並duplicate掉，藉以更新互動摘要
        Posts = pd.concat([Posts, nPost], ignore_index=True)
        Posts = Posts.sort_values(by = 'Updatetime', ascending = False)
        Posts = Posts.drop_duplicates(subset ="Link",
                                      keep= 'first',
                                      inplace = False) 
        Comments = pd.concat([Comments, nComments], ignore_index=True, sort=False)
        print('Result: Succed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
    except:
        print('Result: Failed!')
        print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        print('----------------------------------------------------------')
        print('\n')
Posts.to_csv('C:/Users/TL_Yu/Desktop/Posts.csv',index = False)
Comments.to_csv('C:/Users/TL_Yu/Desktop/Comments.csv',index = False)

Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2439260866113541/
Type: Old Post with New Comments
Result: Succed!
Time Log: 2019-05-15 22:39:22
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2442288712477423/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:39:39
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2449437375095890/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:39:47
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2452941871412107/
Type: New Post
Result: Succed!
Time Log: 2019-05-15 22:40:22
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2451632571543037/
Type: New Post
Result: Succed!
Time Log: 201

Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:45:59
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/posts/2424456250927336
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:46:03
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/posts/2418299714876323
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:46:07
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/posts/2422602024446092
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:46:10
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/posts/2422597434446551
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:46:14
----------------------------------------------------------


Dealing with: https:

Type: Uncertain condiction! Plz Check this website:...
Result: Succed!
Time Log: 2019-05-15 22:48:32
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2392656157440679/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:48:36
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/videos/347523512532780/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:48:40
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2383803504992611/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:48:44
----------------------------------------------------------


Dealing with: https://www.facebook.com/Aptg.tw/photos/a.165348830171434/2383802908326004/
Type: Old Post with Old Comments
Result: Succed!
Time Log: 2019-05-15 22:48:4

In [23]:
gc.collect()

0

In [22]:
datetime.datetime.now().strftime("%Y-%m-%d")

'2019-05-15'

In [23]:
expand('https://www.facebook.com/Aptg.tw/videos/296813794551595/')

In [ ]:
# 嘗試找文章的ID
# 文章的發布時間
# 順著ID連進去這篇文章
# 爬下文章的按讚數、留言數、分享數..等資訊
# 多少個朋友
# 直播影片數量
# 從留言的頭像可以找到發文者的ID


# 每周台灣之星與競業各自再努力些什麼，可以做為例行性的報告
#[Python] [網路爬蟲] 用 selenium 不用登入就能爬過 Facebook
#https://skylinelimit.blogspot.com/2018/09/python-selenium-facebook.html

In [18]:
# 原本想要登入的方式獲取連結，但FB會偵測異常行為，暫時先不登入
# 關閉Chrome的「通知」提醒
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-notifications")

driver = webdriver.Chrome(options=chrome_options)
url = 'https://www.facebook.com/'
driver.get(url)
time.sleep(3)

# 啟動瀏覽器並登入Facebook
username = driver.find_element_by_id('email')
username.send_keys('61034b001@gms.ndhu.edu.tw')
passwd=driver.find_element_by_id('pass')
passwd.send_keys('19920309')
button=driver.find_element_by_id('loginbutton')
button.click()

# Crawl2.0

In [1]:
import pandas as pd
import re, time, requests, datetime, gc
from selenium import webdriver
from bs4 import BeautifulSoup

# 截取塗鴉墻貼文資訊並比對先前資料
如果每次都把所有貼文抓下來，無疑會浪費程式效能，在這裡我將載入先前的資料，並將各篇貼文分成以下三類：

- **新貼文**：截取貼文內容與留言
- **舊貼文但有新留言**：截取所有留言但只將新留言加入資料庫
- **舊貼文也無新留言**：不處理

可想而知，如果設定排程每天執行，大多數資料都會是「舊貼文也無新留言」的類型，加入這個判斷機制將大幅節省系統效能

In [2]:
# 截取塗鴉墻上貼文的留言數
def GetWall_PostCommentCounts(i):
    try:
        CommentCounts = i.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in CommentCounts:
            CommentCounts = int(float(CommentCounts.split('K')[0])*1000)
        else:
            CommentCounts = int(CommentCounts)
    except:
        CommentCounts = 0
    return CommentCounts

# 截取塗鴉墻上貼文的連結
def GetWall_PostLink(i):
    Link = 'https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0]
    return Link

# 截取塗鴉墻上貼文的發佈時間
def GetWall_PostTime(i):
    try:
        Time = i.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = 'Not Post'
    return Time

def CarwlList(urls, n, Posts):
    CheckList = pd.DataFrame(columns = ['Link','Time','CommentCounts']) 
    for i in urls:
        driver.get(i)
        for i in range(n):
            try:
                driver.find_element_by_css_selector('a.pam.uiBoxLightblue.uiMorePagerPrimary').click() # 加載更多貼文的按鈕
            except:
                time.sleep(2)
            # 這裡會跳出要我們登入的大畫面，找到「稍後再說」的按鈕並點擊
            try:
                driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
            except:
                time.sleep(0.5)
            soup = BeautifulSoup(driver.page_source)
            for i in soup.find_all('div', {'class':'_5pcr userContentWrapper'}):
                CheckList = pd.concat([CheckList, pd.DataFrame(data = [{'Link':GetWall_PostLink(i),
                                                                        'Time':GetWall_PostTime(i),
                                                                        'CommentCounts':GetWall_PostCommentCounts(i)}],
                                                                 columns = ['Link','Time','CommentCounts'])],
                                       ignore_index = True)
    
    CarwlList = pd.merge(left = CheckList,
                         right = Posts.loc[:,['Link','CommentCounts']],
                         how='left',
                         on='Link',
                         suffixes=('_c', '_p'),
                         indicator=True)
    CarwlList = pd.concat([CarwlList.loc[CarwlList._merge == 'left_only', :], # 新貼文
                           CarwlList.loc[CarwlList.CommentCounts_c > CarwlList.CommentCounts_p, :]], # 舊貼文但有新留言
                          ignore_index=True)
    CarwlList = CarwlList.drop_duplicates(subset = 'Link',
                                          keep= 'first',
                                          inplace = False)
    return CarwlList

# 爬取特定貼文的內容與留言
## 展開貼文與留言

In [9]:
# 點擊不要現在註冊帳號
def clickNotNow():
    try:
        driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
    except:
        time.sleep(1)

# 將顯示的留言從系統推薦改成所有留言

def SwitchToAllComments():
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    clickNotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    clickNotNow()
    driver.find_element_by_xpath('//div[@class="_5pcr userContentWrapper"]//a[@data-testid="UFI2CommentsCount/root"]').click()
    time.sleep(0.5)
    clickNotNow()
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(0.5)
    clickNotNow()
    driver.find_element_by_xpath('//div[@data-testid="UFI2ViewOptionsSelector/root"]').click()
    clickNotNow()
    driver.find_element_by_partial_link_text('All Comments').click()
    print('SwitchToAllComments Succed!')

# 打開各篇貼文並展開所有留言 與 留言的留言

def PostExpand():
    time.sleep(1)
    # 偵測是否有「更多留言」（第一層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_0"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            clickNotNow()
            try:
                i.click()
            except:
                time.sleep(1)   
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'))>0:
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        for i in driver.find_elements_by_xpath('//a[@data-testid="UFI2CommentsPagerRenderer/pager_depth_1"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            clickNotNow()
            try:
                i.click()
            except:
                time.sleep(1)
    # 偵測是否有「更多留言的留言」（第二層），若有則點擊
    while len(driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'))>0:
        for i in driver.find_elements_by_xpath('//a[@class="_5v47 fss"]'):
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
            # 若彈出註冊視窗點擊not now!
            try:
                i.click()
            except:
                time.sleep(1)

## 爬取貼文內容與留言資訊

In [4]:
# 貼文留言數
def GetPost_PosterName(userContent):
    return userContent.find('img').attrs['aria-label']

# 貼文留言數
def GetPost_PosterID(userContent):
    return userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1]

# 貼文時間
def GetPost_PostTime(userContent):
    try:
        Time = userContent.find('abbr').attrs['title']
        Time = datetime.datetime.strptime(Time, '%m/%d/%y, %I:%M %p')
        Time = Time.strftime("%Y-%m-%d %H:%M")
    except:
        Time = "Error, Please check this post's condiction!" 
    return Time

# 貼文內容
def GetPost_PostContent(userContent):
    try:
        Content = userContent.find('div', {'class':'_5pbx userContent _3576'}).text
    except:
        Content = "There's No Text Content!"
    return Content

# 貼文留言數
def GetPost_PostCommentCounts(userContent):
    try:
        CommentCounts = userContent.find('a', {'data-testid':'UFI2CommentsCount/root'}).text.split(' ',2)[0]
        if 'K' in CommentCounts:
            CommentCounts = int(float(CommentCounts.split('K')[0])*1000)
        else:
            CommentCounts = int(CommentCounts)
    except:
        CommentCounts = 0 
    return CommentCounts

# 貼文分享數
def GetPost_PostShareCounts(userContent):
    try:
        ShareCounts = userContent.find('span', {'class':'_355t _4vn2'}).text.split(' ',2)[0]
        if 'K' in ShareCounts:
            ShareCounts = int(float(ShareCounts.split('K')[0])*1000)
        else:
            ShareCounts = int(ShareCounts)
    except:
        ShareCounts = 0
    return ShareCounts

# 貼文按讚心情數
def GetPost_PostLikeCounts(userContent):
    try:
        LikeCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LIKE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LikeCounts:
            LikeCounts = int(float(LikeCounts.split('K')[0])*1000)
        else:
            LikeCounts = int(LikeCounts)
    except:
        LikeCounts = 0
    return LikeCounts

# 貼文愛心心情數
def GetPost_PostLoveCounts(userContent):
    try:
        LoveCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_LOVE'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in LoveCounts:
            LoveCounts = int(float(LoveCounts.split('K')[0])*1000)
        else:
            LoveCounts = int(LoveCounts)
    except:
        LoveCounts = 0
    return LoveCounts

# 貼文哈哈心情數
def GetPost_PostHahaCounts(userContent):
    try:
        HahaCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_HAHA'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in HahaCounts:
            HahaCounts = int(float(HahaCounts.split('K')[0])*1000)
        else:
            HahaCounts = int(HahaCounts)
    except:
        HahaCounts = 0
    return HahaCounts

# 貼文Wow心情數
def GetPost_PostWowCounts(userContent):
    try:
        WowCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_WOW'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in WowCounts:
            WowCounts = int(float(WowCounts.split('K')[0])*1000)
        else:
            WowCounts = int(WowCounts)
    except:
        WowCounts = 0
    return WowCounts

# 貼文Sad心情數
def GetPost_PostSadCounts(userContent):
    try:
        SadCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_SORRY'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in SadCounts:
            SadCounts = int(float(SadCounts.split('K')[0])*1000)
        else:
            SadCounts = int(SadCounts)
    except:
        SadCounts = 0
    return SadCounts

# 貼文Angry心情數
def GetPost_PostAngryCounts(userContent):
    try:
        AngryCounts = userContent.find('span', {'data-testid':'UFI2TopReactions/tooltip_ANGER'}).find('a').attrs['aria-label'].split(' ',2)[0]
        if 'K' in AngryCounts:
            AngryCounts = int(float(AngryCounts.split('K')[0])*1000)
        else:
            AngryCounts = int(AngryCounts)
    except:
        AngryCounts = 0
    return AngryCounts

# 留言時間
def GetPost_CommentTime(element):
    Time = element.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content']
    Time = datetime.datetime.strptime(Time, '%A, %b %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M")
    return Time

# 留言內容
def GetPost_CommentContent(element):
    try:
        Content = element.find('span', {'dir':'ltr'}).text
    except:
        Content = 'img'
    return Content

# 文章內容與互動摘要
def PostInfo(soup):
    # 貼文區
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    PostContent = pd.DataFrame(data = [{'Name':GetPost_PosterName(userContent),
                                    'ID':GetPost_PosterID(userContent),
                                    'Time':GetPost_PostTime(userContent),
                                    'Content':GetPost_PostContent(userContent),
                                    'CommentCounts':GetPost_PostCommentCounts(userContent),
                                    'Shares':GetPost_PostShareCounts(userContent),
                                    'Like':GetPost_PostLikeCounts(userContent),
                                    'Love':GetPost_PostLoveCounts(userContent),
                                    'Haha':GetPost_PostHahaCounts(userContent),
                                    'Wow':GetPost_PostWowCounts(userContent),
                                    'Sad':GetPost_PostSadCounts(userContent),
                                    'Angry':GetPost_PostAngryCounts(userContent),
                                    'Updatetime':datetime.datetime.now().strftime("%Y-%m-%d %H:%M"),
                                    'Link':driver.current_url,}],
                            columns = ['Name', 'ID', 'Time', 'Content', 'CommentCounts', 'Shares', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Updatetime', 'Link'])
    return PostContent

def PostComments(soup):  
    PostComments = pd.DataFrame()
    userContent = soup.find('div', {'class':'_5pcr userContentWrapper'})
    try:
        for i in userContent.select('ul._7a9a > li'):
            # 先抓留言並放在Comment
            Comment = pd.DataFrame(data=[{'ID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                          'Name':i.find('img').attrs['alt'],
                                          'Time':datetime.datetime.strptime(i.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                          'Content':GetPost_CommentContent(i),
                                          'RepID':userContent.find('div', {'class':'_5pcp _5lel _2jyu _232_'}).attrs['id'].split(';')[0].split('feed_subtitle_')[-1],
                                          'RepName':userContent.find('img').attrs['aria-label'],
                                          'Link':driver.current_url}],
                                   columns = ['ID', 'Name', 'Time', 'Content','RepID','RepName','Link'])
            PostComments = pd.concat([PostComments, Comment], ignore_index=True)
            # 留言的留言
            for j in i.findAll('div', {'data-testid':'UFI2Comment/root_depth_1'}):
                Comment = pd.DataFrame(data=[{'ID':j.find('a', {'class':' _3mf5 _3mg1'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'Name':j.find('img').attrs['alt'],
                                              'Time':datetime.datetime.strptime(j.find('abbr',{'class':'livetimestamp'}).attrs['data-tooltip-content'], '%A, %B %d, %Y at %I:%M %p').strftime("%Y-%m-%d %H:%M"),
                                              'Content':GetPost_CommentContent(j),
                                              'RepID':i.find('a', {'class':' _3mf5 _3mg0'}).attrs['data-hovercard'].split('id=',2)[1],
                                              'RepName':i.find('img').attrs['alt'],
                                              'Link':driver.current_url}],
                                       columns = ['ID', 'Name', 'Time', 'Content','RepID','RepName','Link'])
                PostComments = pd.concat([PostComments, Comment], ignore_index=True)
        PostComments['Updatetime'] = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
    except:
        print('Crawl Comments Failed!')
    return PostComments

# 更新貼文與留言資訊
將最新抓到的貼文併回Post表格，並以ID與Time作為關鍵(key)值，取最新抓取的資料(Updatetime)

In [5]:
def UpdateData(DateFrame_o,DateFrame_n):
    DataFrame = pd.concat([DateFrame_o, DateFrame_n], ignore_index=True)
    DataFrame = DataFrame.sort_values(by = 'Updatetime', ascending = False)
    DataFrame = DataFrame.drop_duplicates(subset = ['ID', 'Time'],
                                          keep= 'first',
                                          inplace = False)
    return DataFrame

# 爬資料
## 讀取先前資料
若在桌面偵測不到檔案，就創造新的dataframe

In [6]:
try:
    Posts = pd.read_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
except:
    Posts = pd.DataFrame(columns=['Name', 'ID', 'Time', 'Content', 'CommentCounts', 'Shares', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Updatetime', 'Link'])
try:
    Comments = pd.read_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')
except:
    Comments = pd.DataFrame(columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName','Link', 'Updatetime'])
    
urls = ['https://www.facebook.com/tstartel/',
        'https://www.facebook.com/chtmobile/',
        'https://www.facebook.com/taiwanmobile/',
        'https://www.facebook.com/fareastone/',
        'https://www.facebook.com/Aptg.tw/']

## 產出待爬清單

In [7]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
# print(driver.get_cookies())
driver.delete_all_cookies()
driver.add_cookie({'domain': '.facebook.com', 'expiry': 1558752322, 'httpOnly': False, 'name': 'locale', 'path': '/', 'secure': True, 'value': 'en_US'})

In [8]:
CarwlList = CarwlList(urls=urls, n=1, Posts = Posts)
CarwlList

,Link,Time,CommentCounts_c,CommentCounts_p,_merge
0,https://www.facebook.com/tstartel/posts/328473...,2019-05-19 20:52,37,NaN,left_only
1,https://www.facebook.com/tstartel/photos/a.413...,2019-05-19 02:00,13,NaN,left_only
2,https://www.facebook.com/tstartel/photos/a.413...,2019-05-18 21:00,55,NaN,left_only
3,https://www.facebook.com/tstartel/photos/a.413...,2019-05-17 21:00,36,NaN,left_only
4,https://www.facebook.com/tstartel/photos/a.854...,2019-05-17 05:10,61,NaN,left_only
5,https://www.facebook.com/tstartel/photos/a.413...,2019-05-17 01:00,69,NaN,left_only
6,https://www.facebook.com/tstartel/posts/327684...,2019-05-16 21:00,18,NaN,left_only
7,https://www.facebook.com/chtmobile/videos/2758...,2019-04-29 21:00,464,NaN,left_only
8,https://www.facebook.com/chtmobile/posts/27207...,2019-05-20 02:00,0,NaN,left_only
9,https://www.facebook.com/chtmobile/photos/a.26...,2019-05-17 02:00,1,NaN,left_only


如同上面的說明，當CheckLists「merge」欄位為「left_only」時表示該貼文是新貼文；

而當「merge」欄位為「both」，但「Comments_c」的值大於「Comments_p」時，表示該貼文是舊資料但有新留言。

以下將截取這兩類的貼文作為爬取清單

## 執行爬蟲

In [10]:
for i in CarwlList.Link:
    print('Dealing with: ' + i)
    driver.get(i)
    try:
        SwitchToAllComments()
        PostExpand()
        try:
            soup = BeautifulSoup(driver.page_source)
            nPost = PostInfo(soup)
            nComments = PostComments(soup)
            try:
                Posts = UpdateData(DateFrame_o = Posts, DateFrame_n = nPost)
                Comments = UpdateData(DateFrame_o = Comments,DateFrame_n = nComments)
                print('Update Succed!')
            except:
                print('Update Posts or Comments Failed!')
        except:
            print('Crawl Post or Comments Failed!')
    except:
        print('No, Comments, Plz Check this post!')
    gc.collect()
    print('Time Log: ' + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + '\n\n------------------')

Dealing with: https://www.facebook.com/tstartel/posts/3284730548219636
SwitchToAllComments Succed!
Update Succed!
Time Log: 2019-05-21 00:06:40

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3275228019169889/
SwitchToAllComments Succed!
Update Succed!
Time Log: 2019-05-21 00:06:56

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3275222502503774/
SwitchToAllComments Succed!
Update Succed!
Time Log: 2019-05-21 00:07:19

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3277738568918834/
SwitchToAllComments Succed!
Update Succed!
Time Log: 2019-05-21 00:07:38

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.854369967922385/3277739692252055/
SwitchToAllComments Succed!
Update Succed!
Time Log: 2019-05-21 00:07:58

------------------
Dealing with: https://www.facebook.com/tstartel/photos/a.413576055335114/3277132165646141/
SwitchToA

C:\Users\TL_Yu\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Update Succed!
Time Log: 2019-05-21 00:08:07

------------------
Dealing with: https://www.facebook.com/tstartel/posts/3276845979008093
SwitchToAllComments Succed!
Update Succed!
Time Log: 2019-05-21 00:08:24

------------------
Dealing with: https://www.facebook.com/chtmobile/videos/2758708034143708/
SwitchToAllComments Succed!
Update Succed!
Time Log: 2019-05-21 00:09:56

------------------
Dealing with: https://www.facebook.com/chtmobile/posts/2720795567950068
No, Comments, Plz Check this post!
Time Log: 2019-05-21 00:10:00

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2722488054447486/
No, Comments, Plz Check this post!
Time Log: 2019-05-21 00:10:07

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/2717455341617424/
SwitchToAllComments Succed!
Update Succed!
Time Log: 2019-05-21 00:10:16

------------------
Dealing with: https://www.facebook.com/chtmobile/photos/a.264830403546609/27152695418

In [11]:
Posts = Posts.reset_index(drop=True)
Posts = Posts.loc[:,['Name', 'ID', 'Time', 'Content', 'CommentCounts', 'Shares', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Updatetime', 'Link']]
Posts

,Name,ID,Time,Content,CommentCounts,Shares,Like,Love,Haha,Wow,Sad,Angry,Updatetime,Link
0,亞太電信Gt智慧生活,162174107155573,2019-05-17 20:00,《三國演義-最強武將傳》亞太電信小額付費代收‼ 最強在此，誰敢來戰😎 史詩級戰爭手遊 《三...,415,322,423,1,0,0,0,0,2019-05-21 00:18,https://www.facebook.com/Aptg.tw/photos/a.1653...
1,亞太電信Gt智慧生活,162174107155573,2019-05-18 04:00,難得的放假不要窩在家 帶著愉悅的好心情，把握難得的好天氣 揪朋友到處走走吃飯聊聊去 #小編隱...,1,0,49,2,0,0,1,0,2019-05-21 00:17,https://www.facebook.com/Aptg.tw/posts/2453373...
2,亞太電信Gt智慧生活,162174107155573,2019-05-18 20:00,週末哪兒走走去❓ 小編推可✅除霉運 ✅指點工作方向 去玩更可以到周邊吃吃N多美食🍴 到底哪兒...,2,0,41,1,0,0,0,0,2019-05-21 00:17,https://www.facebook.com/Aptg.tw/photos/a.1653...
3,亞太電信Gt智慧生活,162174107155573,2019-05-09 21:00,恭喜！亞太電信5月好康3連發，得獎的是….. 等等！這次留給你來決定好惹😤 蒐集義氣胸章抽好...,1500,1000,2900,21,0,0,0,9,2019-05-21 00:16,https://www.facebook.com/Aptg.tw/photos/a.1653...
4,遠傳電信,162583067088337,2019-05-16 20:45,👍遠傳特別企劃 買貴就退差價👍 #文未有好康 想跟職人們一樣用對手機工作生活都Level...,1100,1100,2900,11,0,6,0,0,2019-05-21 00:15,https://www.facebook.com/fareastone/posts/2535...
5,遠傳電信,162583067088337,2019-05-17 20:50,【微上網只要$199 #有省快Buy】 找不到輕量網路的超優方案嗎？ 小編給你真正的 #零錢...,21,7,88,0,0,0,0,0,2019-05-21 00:13,https://www.facebook.com/fareastone/photos/a.1...
6,遠傳電信,162583067088337,2019-05-18 03:30,唱歌最母湯的尷尬時刻！ #留言刷一波 大學生94要夜唱才叫青春啊！ 但你確定你揪的人OK🤔 ...,23,3,52,0,2,1,0,0,2019-05-21 00:13,https://www.facebook.com/fareastone/posts/2538...
7,遠傳電信,162583067088337,2019-05-16 22:00,【什麼巫術讓你一秒回春變寶寶！】 不用冒險進行量子領域的時光旅行 你也可以瞬間變回5歲小朋友...,21,3,54,0,4,0,1,0,2019-05-21 00:13,https://www.facebook.com/fareastone/posts/2539...
8,遠傳電信,162583067088337,2019-05-18 20:45,【尋找10萬旅遊金得主👀 #分享影片抽大獎】 小編眉頭一皺，發現案情不單純！ 遠傳大手筆準備...,104,41,127,1,0,0,0,0,2019-05-21 00:12,https://www.facebook.com/fareastone/posts/2538...
9,遠傳電信,561030511088422:9:0,2019-05-19 20:45,🙋♂打敗遠距💗有話慢慢講 讓愛遠遠傳🙋♀ #遠距離戀愛法則大公開 有人覺得 最美的愛有距...,15,4,45,0,0,0,0,0,2019-05-21 00:12,https://www.facebook.com/fareastone/videos/561...


In [12]:
Comments = Comments.reset_index(drop=True)
Comments = Comments.loc[:,['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName','Link', 'Updatetime']]
Comments

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,162174107155573,亞太電信Gt智慧生活,2019-05-20 11:31,板橋車站附近也蠻多好吃的說～,100001931546454,戴豊基,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-21 00:17
1,100008036701674,陳國進,2019-05-19 06:33,看起來好好吃速度也吃的好多,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/posts/2453373...,2019-05-21 00:17
2,100001931546454,戴豊基,2019-05-19 11:19,板橋車站東1門出口比較好！,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-21 00:17
3,100000493953848,Olivia Wang,2019-05-19 00:43,「義氣讚出來黃珊和我一起到亞太電信門市將SHARP 40吋電視搬回家」,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-21 00:16
4,100009419714822,吳久生,2019-05-12 21:56,「義氣讚出來 吳玉葉 和我一起到亞太電信門市將SHARP 40吋電視搬回家」。,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-21 00:16
5,100004206399097,姜業樑,2019-05-11 19:15,「義氣讚出來 Le Ybinh 和我一起到亞太電信門市將SHARP 40吋電視搬回家」。,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-21 00:16
6,100014322184237,Yi En Lin,2019-05-11 12:50,義氣讚出來 蕭世筠 和我一起到亞太電信門市將SHARP 40吋電視搬回家,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-21 00:16
7,100003303399116,黃荷珺,2019-05-10 12:55,義氣讚出來 Hanxian Huang 和我一起到亞太電信門市將SHARP 40吋電視搬回家,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-21 00:16
8,100012941294896,蕭蕭,2019-05-11 14:23,「義氣讚出來 Apple Iria和我一起到亞太電信門市將SHARP 40吋電視搬回家」,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-21 00:16
9,100014852358781,蔡明哲,2019-05-14 14:31,義氣讚出來 Katherine Tsai和我一起到亞太電信門市將SHARP 40吋電視搬回家,162174107155573,亞太電信Gt智慧生活,https://www.facebook.com/Aptg.tw/photos/a.1653...,2019-05-21 00:16


In [27]:
Posts.to_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
Comments.to_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')

In [28]:
Posts.to_excel('C:/Users/TL_Yu/Desktop/Posts.xlsx')
Comments.to_excel('C:/Users/TL_Yu/Desktop/Comments.xlsx')

# 維護測試區

In [26]:
try:
    Posts = pd.read_pickle('C:/Users/TL_Yu/Desktop/Posts.plk')
except:
    Posts = pd.DataFrame(columns=['Name', 'ID', 'Time', 'Content', 'CommentCounts', 'Shares', 'Like', 'Love', 'Haha', 'Wow', 'Sad', 'Angry', 'Updatetime', 'Link'])
try:
    Comments = pd.read_pickle('C:/Users/TL_Yu/Desktop/Comments.plk')
except:
    Comments = pd.DataFrame(columns = ['ID', 'Name', 'Time', 'Content', 'RepID', 'RepName','Link', 'Updatetime'])
    
urls = ['https://www.facebook.com/tstartel/',
        'https://www.facebook.com/chtmobile/',
        'https://www.facebook.com/taiwanmobile/',
        'https://www.facebook.com/fareastone/',
        'https://www.facebook.com/Aptg.tw/']

In [27]:
driver = webdriver.Chrome()
driver.get('https://www.facebook.com/')
# print(driver.get_cookies())
driver.delete_all_cookies()
driver.add_cookie({'domain': '.facebook.com', 'expiry': 1558752322, 'httpOnly': False, 'name': 'locale', 'path': '/', 'secure': True, 'value': 'en_US'})

In [124]:
https://www.facebook.com/tstartel/photos/a.854369967922385/3277739692252055/

a
b
c
d
e
f
Expand Post Comments Succed!a


In [11]:
driver.get('https://www.facebook.com/tstartel/photos/a.413576055335114/3277132165646141/')

In [12]:
SwitchToAllComments()

SwitchToAllComments Succed!


In [13]:
PostExpand()

In [52]:
soup = BeautifulSoup(driver.page_source)
nPost = PostInfo(soup)
nComments = PostComments(soup)

In [53]:
nPost

,Name,ID,Time,Content,CommentCounts,Shares,Like,Love,Haha,Wow,Sad,Angry,Updatetime,Link
0,台灣之星,360044337354953,2019-05-18 21:00,好想去日本🇯🇵看繡球花唷 小編們聽 #U姐編 嘟嘟囔囔一週快煩死了 交換個眼色，作戰計劃開始...,55,13,626,8,2,0,0,0,2019-05-20 23:47,https://www.facebook.com/tstartel/photos/a.413...


In [54]:
nComments

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
0,100034741457030,吳世凱,2019-05-19 22:23,真的很美適合情侶照或結婚照,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
1,360044337354953,台灣之星,2019-05-19 22:32,大推~❤❤,100034741457030,吳世凱,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
2,100035760513977,蔡寶淑,2019-05-20 11:43,好漂亮的花海,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
3,360044337354953,台灣之星,2019-05-20 11:54,台灣就是如此美麗~幸福❤,100035760513977,蔡寶淑,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
4,100008036701674,陳國進,2019-05-19 15:32,不錯~~,360044337354953,台灣之星,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
5,360044337354953,台灣之星,2019-05-19 15:40,賞心悅目~心情好❤🌼,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
6,100008036701674,陳國進,2019-05-19 15:44,台灣之星 我現在在潮州~~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
7,360044337354953,台灣之星,2019-05-19 15:52,看美美照片一樣心情會hen好,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
8,100008036701674,陳國進,2019-05-19 15:52,台灣之星 是啊～給大心,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
9,360044337354953,台灣之星,2019-05-19 15:53,img,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47


In [55]:
Posts = UpdateData(DateFrame_o = Posts, DateFrame_n = nPost)
Posts

,Name,ID,Time,Content,CommentCounts,Shares,Like,Love,Haha,Wow,Sad,Angry,Updatetime,Link
2,台灣之星,360044337354953,2019-05-18 21:00,好想去日本🇯🇵看繡球花唷 小編們聽 #U姐編 嘟嘟囔囔一週快煩死了 交換個眼色，作戰計劃開始...,55,13,626,8,2,0,0,0,2019-05-20 23:47,https://www.facebook.com/tstartel/photos/a.413...
0,台灣之星,360044337354953,2019-05-19 02:00,#週1福利日 #每週一1230 #週週登場 有沒有人跟 #00C編 一樣 手機沒電會狂症發作...,13,4,299,3,0,5,0,0,2019-05-20 23:46,https://www.facebook.com/tstartel/photos/a.413...
1,台灣之星,360044337354953,2019-05-19 20:52,反正只要一兩塊錢 買個袋子裝一裝，比較方便啦😆 環保餐具用完還要洗，好麻煩 直接拿竹筷和塑膠...,35,13,79,1,0,0,3,0,2019-05-20 23:45,https://www.facebook.com/tstartel/posts/328473...


In [56]:
Comments = UpdateData(DateFrame_o = Comments,DateFrame_n = nComments)
Comments

,ID,Name,Time,Content,RepID,RepName,Link,Updatetime
48,360044337354953,台灣之星,2019-05-19 15:52,看美美照片一樣心情會hen好,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
61,360044337354953,台灣之星,2019-05-19 19:40,img,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
70,100008036701674,陳國進,2019-05-20 17:18,台灣之星 還好我16:00回到家過一下才下雨~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
69,360044337354953,台灣之星,2019-05-20 16:48,有沒有記得帶雨傘呀~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
68,100008036701674,陳國進,2019-05-20 16:44,台灣之星 小編，下大雨了,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
67,360044337354953,台灣之星,2019-05-19 21:08,加油加油(๑˃̵ᴗ˂̵)ﻭ其實上課是很幸福D~~晚安😉😉,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
66,100008036701674,陳國進,2019-05-19 21:03,台灣之星 明天又要上課，小編晚安~~,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
65,360044337354953,台灣之星,2019-05-19 20:47,早點洗洗睡明天又是新的一天~(✿◠‿◠),100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
64,100008036701674,陳國進,2019-05-19 20:42,台灣之星 是啊～,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
63,360044337354953,台灣之星,2019-05-19 20:41,國進到家啦❓,100008036701674,陳國進,https://www.facebook.com/tstartel/photos/a.413...,2019-05-20 23:47
